In [1]:
import numpy as np
import os
print(os.getcwd())

# If your loaded arrays have only 10,000 test cases, set num_test_cases accordingly
num_test_cases = 10000
dim0 = 16

test_cases_act = np.load("test_cases_act.npy")
test_cases_w = np.load("test_cases_w.npy")

# Ensure that num_test_cases does not exceed the size of the loaded arrays
num_test_cases = min(num_test_cases, test_cases_act.shape[0], test_cases_w.shape[0])

for i in range(num_test_cases):
    in1 = test_cases_act[i]
    in2 = test_cases_w[i]
    # Process the test cases as needed


/root/jupyter_notebooks/getting_started


In [2]:
in1_py = np.resize(in1, (16,16))
in2_py = np.resize(in2, (16,16))

In [3]:
import time
start_time = time.time()
out_py = np.dot(in1_py,in2_py)
end_time = time.time()
python_time = end_time - start_time
print("SW matrix mul exe time: {}s".format(end_time - start_time))

SW matrix mul exe time: 0.0010807514190673828s


In [4]:
#print("the result of software matmul: ", out_py)

In [5]:
from pynq import Overlay
overlay_baseline = Overlay("./matmul_plain/matmul_plain.bit")
baseline = overlay_baseline.matmul_plain_0

In [6]:
from pynq import allocate
input1_buffer0 = allocate(shape=(dim0*dim0,), dtype='f4')
input2_buffer0 = allocate(shape=(dim0*dim0,), dtype='f4')
output_buffer0 = allocate(shape=(dim0*dim0,), dtype='f4')

In [7]:
np.copyto(input1_buffer0, in1.flatten())
np.copyto(input2_buffer0, in2.flatten())

print("input1_buffer0:", input1_buffer0)
print("input2_buffer0:", input2_buffer0)
print("output_buffer0:", output_buffer0)

input1_buffer0: [0.73258775 0.42161947 0.9020244  0.8986483  0.78794223 0.25514364
 0.23724885 0.53089744 0.68482655 0.22423759 0.20156232 0.5279975
 0.04716639 0.29832137 0.8445518  0.85050917 0.2926288  0.03918346
 0.63519675 0.7788084  0.48864377 0.1419093  0.92610705 0.28050676
 0.49248293 0.10705224 0.74916106 0.41053647 0.15549912 0.4712087
 0.8641846  0.7968396  0.5666338  0.68381757 0.23333012 0.47867492
 0.16158284 0.83462304 0.15489921 0.20713195 0.1558855  0.6116028
 0.6919567  0.10319018 0.35053527 0.5213497  0.34873527 0.21198912
 0.81576777 0.3908266  0.63780034 0.08257091 0.27531126 0.2598942
 0.0721826  0.15766999 0.06302351 0.34831658 0.1282713  0.85769606
 0.07600531 0.74245137 0.09749908 0.8396064  0.9345244  0.5828034
 0.67200524 0.8318054  0.89329636 0.3006516  0.20478983 0.39395106
 0.9941717  0.7555281  0.93751127 0.87097245 0.97554404 0.40191692
 0.17187695 0.98241276 0.70320606 0.28829572 0.742818   0.37740532
 0.36090764 0.06344465 0.05040553 0.92025787 0.9447

In [8]:
input1_buffer0.sync_to_device()
input2_buffer0.sync_to_device()

#print("input1_buffer0:", input1_buffer0)
#print("input2_buffer0:", input2_buffer0)

In [9]:
#baseline.register_map

In [10]:
baseline.register_map.B_1.address

28

In [11]:
# Write A address (low and high)
baseline.write(baseline.register_map.A_1.address, input1_buffer0.physical_address & 0xFFFFFFFF)
baseline.write(baseline.register_map.A_2.address, (input1_buffer0.physical_address >> 32) & 0xFFFFFFFF)

# Write B address (low and high)
baseline.write(baseline.register_map.B_1.address, input2_buffer0.physical_address & 0xFFFFFFFF)
baseline.write(baseline.register_map.B_2.address, (input2_buffer0.physical_address >> 32) & 0xFFFFFFFF)

# Write AB address (low and high)
baseline.write(baseline.register_map.AB_1.address, output_buffer0.physical_address & 0xFFFFFFFF)
baseline.write(baseline.register_map.AB_2.address, (output_buffer0.physical_address >> 32) & 0xFFFFFFFF)

In [12]:
input1_buffer0.sync_to_device()
input2_buffer0.sync_to_device()

# Start the IP
baseline.write(0x00, 0x01)

# Wait for AP_DONE
while (baseline.read(0x00) & 0x2) == 0:
    pass

output_buffer0.sync_from_device()

In [13]:
baseline.write(0x00, 0x01)
start_time = time.time()
while True:
    reg = baseline.read(0x00)
    if reg != 1:
        break
end_time = time.time()
baseline_time = end_time - start_time
print("HW mul (baseline) exe time: {}s".format(end_time - start_time))

HW mul (baseline) exe time: 0.00043582916259765625s


In [14]:
output_buffer0.sync_from_device()
print("output_buffer0:", output_buffer0)

output_buffer0: [4.8102717 3.9217894 5.132691  4.6050296 4.5287085 5.340448  3.8236003
 3.9983904 3.6727695 3.9528456 4.1126537 3.9720562 4.41327   5.142331
 4.5084124 3.6976428 3.7613509 4.106799  4.9053955 4.2977633 3.8692098
 4.4057393 4.0846753 3.2208097 2.887535  3.4517083 3.8271232 3.9135063
 4.13209   4.9959507 3.5172555 3.2260115 2.8624172 2.9675145 4.0182962
 3.7964532 3.1431787 4.2661815 2.785727  3.7948215 3.0533562 2.863937
 2.754157  3.1889634 3.2126586 3.694215  3.2379382 3.1428304 2.6624846
 2.306399  3.6034825 2.6692772 3.0348558 3.9555354 2.5433774 3.209328
 2.3495157 2.6573424 2.753684  2.5905492 2.7310963 3.4791842 2.6934347
 2.3459604 5.9534717 5.5697474 6.9010973 6.2199    5.9069333 6.994398
 5.2620955 5.306686  5.074883  4.5907574 5.4828176 5.287881  5.334689
 6.2122846 5.1094    4.5031753 4.0473785 3.9724028 4.483434  3.437607
 3.5014706 5.0684705 2.9449503 4.060342  3.3307111 2.5232434 3.1863253
 3.787863  3.24139   3.9926827 4.3755865 2.6852753 2.9080637 2.6691

In [15]:
output_buffer0

PynqBuffer([4.8102717, 3.9217894, 5.132691 , 4.6050296, 4.5287085,
            5.340448 , 3.8236003, 3.9983904, 3.6727695, 3.9528456,
            4.1126537, 3.9720562, 4.41327  , 5.142331 , 4.5084124,
            3.6976428, 3.7613509, 4.106799 , 4.9053955, 4.2977633,
            3.8692098, 4.4057393, 4.0846753, 3.2208097, 2.887535 ,
            3.4517083, 3.8271232, 3.9135063, 4.13209  , 4.9959507,
            3.5172555, 3.2260115, 2.8624172, 2.9675145, 4.0182962,
            3.7964532, 3.1431787, 4.2661815, 2.785727 , 3.7948215,
            3.0533562, 2.863937 , 2.754157 , 3.1889634, 3.2126586,
            3.694215 , 3.2379382, 3.1428304, 2.6624846, 2.306399 ,
            3.6034825, 2.6692772, 3.0348558, 3.9555354, 2.5433774,
            3.209328 , 2.3495157, 2.6573424, 2.753684 , 2.5905492,
            2.7310963, 3.4791842, 2.6934347, 2.3459604, 5.9534717,
            5.5697474, 6.9010973, 6.2199   , 5.9069333, 6.994398 ,
            5.2620955, 5.306686 , 5.074883 , 4.5907574, 5.4828

In [16]:
out_hw = output_buffer0.reshape((dim0, dim0))

#print("Hardware result:\n", out_hw)
#print("Software result:\n", out_py)

# Compare results
diff = np.abs(out_hw - out_py)
max_diff = np.max(diff)
print(f"Test Case {test_cases_act + 1}: Max Difference = {max_diff}")

Test Case [[[1.3434633 1.558025  1.3455644 ... 1.1150253 1.8604597 1.367887 ]
  [1.2454884 1.6404096 1.8628209 ... 1.8297216 1.6574447 1.9102745]
  [1.3804984 1.387207  1.1508131 ... 1.6957198 1.399256  1.6701918]
  ...
  [1.8044143 1.6801121 1.191068  ... 1.443607  1.8813312 1.6045249]
  [1.0467243 1.0561469 1.4479126 ... 1.7814697 1.9329374 1.3467238]
  [1.4742644 1.6534886 1.4733211 ... 1.5842919 1.7854229 1.5777628]]

 [[1.5660732 1.538767  1.6186962 ... 1.2120614 1.648307  1.8688645]
  [1.702558  1.1812612 1.9493358 ... 1.9005835 1.841887  1.1192684]
  [1.7870965 1.8400434 1.6545737 ... 1.0801747 1.0378364 1.9579101]
  ...
  [1.6758854 1.1034436 1.6444622 ... 1.2010831 1.4274566 1.8552775]
  [1.7811118 1.3672612 1.8168514 ... 1.7015853 1.4420466 1.7298882]
  [1.7647616 1.1876491 1.2248838 ... 1.714009  1.9863789 1.1051716]]

 [[1.3048182 1.6726463 1.1282498 ... 1.9485214 1.221036  1.4999357]
  [1.6497495 1.9317031 1.7309761 ... 1.4395924 1.7989652 1.5414236]
  [1.1216197 1.9507926